In [22]:
## NAME:Devyani Mardia
## NETID: dm1633

################
## Code for HW 2 problem 1
##
## INSTRUCTIONS:
## The following file implements logistic lasso:
##    min_{beta}  \sum_{i=1}^n - Y_i (X_i^T beta) + log(1 + exp(X_i^T beta))   + lambda * |beta|_1
##
## We use proximal gradient descent, i.e., ISTA but with a different gradient.
## 
## Fill in the code in parts labeled "FILL IN". There are FIVE parts that you have to fill in.
##
################

import numpy as np


In [23]:

## logistic lasso
## INPUT: X  n--by--p matrix, Y n--by--1 vector, lambda scalar
## OUTPUT: beta p--by--1 vector
##
def logisticLasso(X, Y, lambd):
    p = X.shape[1]
    n = X.shape[0]
    
    beta = np.ones(p)/p
    stepsize = 1
    
    # parameter for backtracking line search
    alpha = 0.05
    gamma = 0.8
    
    it = 0
    while True:
        cur_obj = logisticLassoObj(X, Y, beta, lambd)
        
        xb = X @ beta
        phi = np.exp(xb)/(1 + np.exp(xb))
        
        # compute gradient update
        gradient = X.T @ (phi - Y)
        ## FILL IN: compute the gradient
        beta_new = softThresh(beta - stepsize*gradient, lambd*stepsize)## FILL IN: compute the new beta
        
        # backtracking line search
        while (logisticLassoObj(X, Y, beta_new, lambd) > cur_obj + alpha * np.sum( (beta_new - beta)*gradient ) ):
            stepsize = stepsize * gamma
            beta_new = softThresh(beta - stepsize*gradient, lambd*stepsize)
        
        it = it+1

        if it % 2 == 0:
            print("iteration: %d   objective: %.4f   stepsize (log10): %.4f" % 
                  (it, logisticLassoObj(X, Y, beta_new, lambd), np.log10(stepsize)))
            
        if np.sum((beta - beta_new)**2)/np.sum(beta**2) < 1e-10:
            return beta_new
        else:
            beta = beta_new


In [24]:
def softThresh(u, lambd):
    u[abs(u) <= lambd] = 0
    u[u > lambd] = u[u > lambd] - lambd
    u[u < -lambd] = u[u < -lambd] + lambd
    return u


## INPUT: vector x
## OUTPUT: vector of e^x/(1 + e^x)
def sigmoid(x):
    return np.exp(x)/(1 + np.exp(x))

In [25]:
## OUTPUT: objective of the logistic lasso loss, a single scalar
def logisticLassoObj(X, Y, beta, lambd):
    
    obj = np.sum(-Y * (X @ beta) + np.log(1 + np.exp(X @ beta))) + lambd * np.linalg.norm(beta, ord=1)
                ## FILL IN: compute the logistic lasso objective
                ##        \sum_{i=1}^n - Y_i (X_i^T beta) + log(1 + exp(X_i^T beta))  + lambda * |beta|_1
                ##          with respect to X, Y, beta, lambda
    
    return obj


In [26]:
###############
###############
## 
## Testing our algorithm
## 
###############
###############

from sklearn.linear_model import LogisticRegression

np.random.seed(1)

p = 100
n = 500
s = 5

beta = np.concatenate((np.random.normal(size=s), np.zeros(p-s)))

X = np.random.normal(size=(n, p))
Y = np.sign(X @ beta + np.random.normal(size=n) * 0.3)
Y[Y == -1] = 0

lambd = 0.05

betahat = logisticLasso(X, Y, lambd * n)

clf = LogisticRegression(penalty='l1', solver='liblinear', C=1/(n*lambd), fit_intercept=False)
clf.fit(X, Y)
betahat2 = clf.coef_.ravel()

estimation_err = np.sum((betahat - beta)**2)/np.sum(beta**2)
dist_to_skl = np.sum((betahat2 - betahat)**2)/np.sum(betahat2**2)

print("Estimation error: %.4f   Deviation from Sklearn solution: %.4f" % (estimation_err, dist_to_skl))


iteration: 2   objective: 254.7932   stepsize (log10): -1.5506
iteration: 4   objective: 235.3171   stepsize (log10): -1.5506
iteration: 6   objective: 235.1317   stepsize (log10): -1.7444
iteration: 8   objective: 235.0110   stepsize (log10): -1.8413
iteration: 10   objective: 235.0030   stepsize (log10): -2.8104
Estimation error: 0.0888   Deviation from Sklearn solution: 0.0000
